# Analyzing Image data
In this session, we'll continue our analysis of orientation and motion tuning in the calcium imaging sample from mouse V1. Our first step will be to refactor our current tuning analysis into a function, making it easier to swap out our method for determining preferred orientation while keeping the rest of our code the same (aka, the [strategy pattern](https://en.wikipedia.org/wiki/Strategy_pattern)). The second step will be to write code that calculates tuning across the entire image.

```{important}
In this lesson, we will only be considering the simplest type of analysis &mdash; analyzing images at the pixel level. For any serious analysis, you would want to use packages like [Suite2p](https://www.suite2p.org) or [CaImAn](https://caiman.readthedocs.io/en/master/) that detect regions of interest (ROIs), typically cell bodies, and extract their time courses.
```

## Tuning curves: refactoring
Before we start, let's clean up our code:

```{admonition} Exercise
1. Rewrite your code for finding the preferred stimulus as a function.
1. Write code that finds the preferred orientation for each pixel in the image.
1. Display the result as a color-mapped image:
    - To get a colormap with `n` values, use, e.g., `cm.get_cmap('jet', n)` (explanations [here](https://matplotlib.org/tutorials/colors/colormap-manipulation.html) and [here](https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html)).
    - To set the colormap for the current image, use `cmap=<cmap name>`, where `<cmap name>` is a colormap name. There are [lots to choose from](https://matplotlib.org/stable/tutorials/colors/colormaps.html).
    - You will want to create a discrete color axis by assigning each value in the array to a color. This is illustrated in [PDSH](https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html#Discrete-Color-Bars).
1. What do you note about the resulting image? What does or doesn't make sense biologically?
```

```{warning}
The image can take a while to plot (> 1 minute on a decent machine), so you may want to settle for plotting a smaller section of the image while you debug.
```

### Solution:

In [ ]:
!gdown 10mbjgJIkyaczKs47_f4es0M3AI7gqTy3

In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sci

vars = sci.loadmat('DirectionTuning_V1_dec.mat')

data = vars['data']
dirTuningExp = vars['dirTuningExp']

Nvert, Nhorz, Nframes = data.shape
dt = 1/3  # 3 Hz sampling 

labels = -1 * np.ones([Nframes, 1])
codes = dirTuningExp['tGratingDirectionDeg'][0][0][0]
Ntrials = dirTuningExp['nTrials'][0][0][0][0]
Noff = dirTuningExp['stimOffFrames'][0][0][0][0]
Non = dirTuningExp['stimOnFrames'][0][0][0][0]

offset = 0
for i in range(Ntrials):
    offset = offset + Noff
    labels[offset + np.array(range(Non))] = codes[i]
    offset = offset + Non

In [ ]:
# write a function that repeats what we did last time
# for a single pixel

def find_pixel_tuning(tseries, labels):
    """
    Given a time series and labels (-1 = baseline) for the stimulus shown in each 
    frame, return the preferred direction of the stimulus as an index
    into the categories of labels
    """
    
    dat = pd.DataFrame(data = {'orientation': labels.ravel(), 'trace': tseries})

    # convert labels to categorical variable
    dat['orientation'] = dat['orientation'].astype('category')
    
    ## group stats
    tuning = dat.groupby(by = 'orientation').mean()
    
    # find the max absolute response relative to baseline
    baseline = tuning.iloc[0].values
    tcurve = tuning.iloc[1:].values - baseline
    return np.argmax(tcurve)

In [ ]:
# now let's repeat for all pixels
preferred_img = np.full((Nvert, Nhorz), np.nan)
for hh in range(Nhorz):
    for vv in range(Nvert):
        preferred_img[vv, hh] = find_pixel_tuning(data[vv, hh, :], labels)

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']

orientations = np.unique(labels[labels != -1])

# plot preferred image
from matplotlib import cm
cmap = cm.get_cmap('jet', orientations.shape[0])
cax = plt.imshow(preferred_img, cmap=cmap, aspect='auto')
cbar = plt.colorbar(cax)
cbar.ax.set_yticks(np.unique(preferred_img))
cbar.ax.set_yticklabels(orientations.astype(str));

For the rest of this exercise, we'll be improving our methods of calculating and plotting the pixel tuning.

## Tuning curves: handling noise
If we want to consider our pixel as tuned, or a particular value the preferred value, it will help to have a sense of how confident we can be in our tuning curve.

```{admonition} Exercise
1. Modify your code to calculate both the mean and the standard error of the mean for both the baseline and each stimulus.
    - How do the standard errors for the stimulus and baseline means compare? Why?
    - Is the standard error a good measure of variability for this type of data?
    - Assuming the standard error is okay, are we over- or under-estimating the variability of the signal? How might we change this?
1. How might you test whether a particular value is really a peak, or just a fluctuation due to noise? There are multiple possible strategies. Discuss with me and then implement this change in a new function. Make sure not to use any [magic numbers](https://en.wikipedia.org/wiki/Magic_number_(programming)#Unnamed_numerical_constants). That is, any parameters of your algorithm should be specifiable by callers of your function.
1. Re-plot the tuning map.
```

```{hint}
When considering the difference of two Gaussian variables $X$ and $Y$, the variance of their _difference_ is $\sigma^2_{X - Y} = \sigma^2_X + \sigma^2_Y$
```

### Solution:

In [ ]:
# wrap the codes as a function
def find_pixel_tuning2(tseries, labels, thresh):
    """
    Given a time series and labels (-1 = baseline) for the stimulus shown in each 
    frame, return the preferred direction of the stimulus as an index
    into the categories of labels
    """
    
    dat = pd.DataFrame(data = {'orientation': labels.ravel(), 'trace': tseries})

    # convert labels to categorical variable
    dat['orientation'] = dat['orientation'].astype('category')
    
    ## group stats
    tuning = dat.groupby(by = 'orientation').mean()
    sems = dat.groupby(by = 'orientation').sem()
    
    # calculate get means and standard errors of the means for 
    # both baseline and stimulus values
    baseline = tuning.iloc[0].values
    bsem = sems.iloc[0].values

    tcurve = tuning.iloc[1:].values - baseline
    tsems = sems.iloc[1:].values

    peakind = np.argmax(tcurve)
    
    # and let's be sure this is > 'thresh' standard deviations from baseline
    threshold = thresh * np.sqrt(tsems[peakind] ** 2 + bsem ** 2)
    if np.abs(tcurve[peakind]) > threshold:
        preferred = peakind
    else:
        preferred = np.nan
        
    return preferred


```{toggle}
Note that sem is a bad measure here for two reasons:
1. The signal is only positive (or should be: you can't have negative
light), so it's not normally distributed. Standard errors are based on
normality assumptions.
2. The sem calculation treats every frame as independent, which is
clearly wrong. We could do better by averaging frames within trials and
calculating the sem across trials, since those numbers are more
independent.
```

In [ ]:
# now let's repeat for all pixels
preferred_img = np.full((Nvert, Nhorz), np.nan)
for hh in range(Nhorz):
    for vv in range(Nvert):
        preferred_img[vv, hh] = find_pixel_tuning2(data[vv, hh, :], labels, 3)

In [ ]:
# plot preferred image
cax = plt.imshow(preferred_img, cmap=cmap, aspect='auto')
cbar = plt.colorbar(cax)
cbar.ax.set_yticks(np.unique(preferred_img[~np.isnan(preferred_img)]))
cbar.ax.set_yticklabels(orientations.astype(str));

## Tuning curves: tuned at all?
It's clear from both biology and the data at hand that not every pixel is even responsive to the stimuli. We would like a way to "opt out" of coloring a pixel with a preferred orientation in cases where that doesn't make sense.

```{admonition} Exercise
1. What test would you use to determine whether or not a pixel is tuned at all? After discussing with me, implement this change in your code?
    - This does not need to be a rigorous statistical test (though it could be).
    - How will you indicate "not tuned" in the return value from your function?
1. Re-plot the data. Make sure your colors reflect that some pixels have no apparent tuning.
```

```{hint}
You can use `cmap.set_bad('white')` to plot NaN values in the resulting image as white.
```

### Solution:

In [ ]:
def find_pixel_tuning3(tseries, labels, min_baseline, thresh):
    # given a time series and labels (converted to categorical later) of the stimulus shown in each frame (-1 =
    # baseline), return the preferred direction of the stimulus as an index
    # into the categories of labels
    
    dat = pd.DataFrame(data={'orientation': labels.ravel(), 'trace': tseries})
    dat['orientation'] = dat['orientation'].astype('category')
    
    ## group stats
    tuning = dat.groupby(by = 'orientation').mean()
    sems = dat.groupby(by = 'orientation').sem()

    baseline = tuning.iloc[0].values
    bsem = sems.iloc[0].values

    tcurve = tuning.iloc[1:].values - baseline
    tsems = sems.iloc[1:].values

    peakind = np.argmax(tcurve)
    
    # now we want two conditions:
    # 1. baseline needs to be greater than some minimum
    # 2. peak activity needs to be more than some given number of standard
    #    deviations above that
    threshold = thresh * np.sqrt(tsems[peakind] ** 2 + bsem ** 2)
    if baseline > min_baseline and np.abs(tcurve[peakind]) > threshold:
        preferred = peakind
    else:
        preferred = np.nan
        
    return preferred


In [ ]:
# now let's repeat for all pixels
preferred_img = np.full((Nvert, Nhorz), np.nan)
for hh in range(Nhorz):
    for vv in range(Nvert):
        preferred_img[vv, hh] = find_pixel_tuning3(data[vv, hh, :], labels, 2e4, 3)

In [ ]:
cmap.set_bad('black')
cax = plt.imshow(preferred_img, cmap=cmap, aspect='auto')
cbar = plt.colorbar(cax)
cbar.ax.set_yticks(np.unique(preferred_img[~np.isnan(preferred_img)]))
cbar.ax.set_yticklabels(orientations.astype(str));

In [ ]:
import os
os.remove('DirectionTuning_V1_dec.mat')